In [ ]:
%load_ext autoreload
%autoreload

import matplotlib.pyplot as plt
import pandas as pd
from pipeline import *
from model import *
from bucket import *

figure = plt.figure(figsize=(20,11))

nblocks = 11
nticks = 10

SUBMISSION_OVERHEAD = 1
MAX_IOPS = 10000
BASE_COMPLETION_LATENCY = 1.2
CONSUMPTION_RATE = 2

completion_target_distance = 512
min_dispatch = 2
max_inflight = 10

pipeline = Pipeline(nblocks,
                    completion_target_distance, min_dispatch, max_inflight, 
                    SUBMISSION_OVERHEAD, MAX_IOPS, BASE_COMPLETION_LATENCY, CONSUMPTION_RATE)


pipeline.run(nticks, nblocks)
to_plot = pipeline.measure(nticks)

df = pd.DataFrame(to_plot)
df.plot(figsize=(20,11), secondary_y='consumed')